Imports

In [16]:
import requests
import pandas as pd
import sqlite3
import os


Function

In [ ]:
def get_temperature_data(city_name, latitude, longitude, year):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={latitude}"
        f"&longitude={longitude}"
        f"&start_date={year}-01-01"
        f"&end_date={year}-12-31"
        f"&daily=temperature_2m_max"
        f"&timezone=Europe%2FBerlin"
    )
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        dates = data['daily']['time']
        temps = data['daily']['temperature_2m_max']
        return {
            "city": city_name,
            "year": year,
            "dates": dates,
            "temperatures": temps
        }
    else:
        print("Failed to retrieve data:", response.status_code)
        return None


Define Cities and Years

In [28]:
cities = [
    {"name": "Zurich", "lat": 47.37, "lon": 8.55},
    {"name": "Rome", "lat": 41.89, "lon": 12.49},
    {"name": "London", "lat": 51.51, "lon": -0.13}
]
years = list(range(2014, 2019))

Fetch data from API and save locally to avoid API spamming

In [30]:
import os
import json
import time

records = []

# Ensure data folder exists
os.makedirs("data", exist_ok=True)

for city in cities:
    for year in years:
        filename = f"data/{city['name']}_{year}.json"

        # Try to load from cache
        if os.path.exists(filename):
            print(f"Loading cached data for {city['name']} {year}")
            with open(filename, "r") as f:
                result = json.load(f)
        else:
            print(f"Fetching data for {city['name']} {year}")
            result = get_temperature_data(city["name"], city["lat"], city["lon"], year)
            if result:
                with open(filename, "w") as f:
                    json.dump(result, f)
            time.sleep(1.2)  # avoid API spamming

        if result:
            avg_temp = sum(result["temperatures"]) / len(result["temperatures"])
            records.append({
                "City": result["city"],
                "Year": result["year"],
                "AvgMaxTemp": round(avg_temp, 2)
            })


Loading cached data for Zurich 2014
Loading cached data for Zurich 2015
Loading cached data for Zurich 2016
Loading cached data for Zurich 2017
Loading cached data for Zurich 2018
Loading cached data for Rome 2014
Loading cached data for Rome 2015
Loading cached data for Rome 2016
Loading cached data for Rome 2017
Loading cached data for Rome 2018
Fetching data for London 2014
Fetching data for London 2015
Fetching data for London 2016
Fetching data for London 2017
Fetching data for London 2018


Collect and Aggregate Data

In [32]:
import time

records = []

for city in cities:
    for year in years:
        result = get_temperature_data(city["name"], city["lat"], city["lon"], year)
        if result:
            avg_temp = sum(result["temperatures"]) / len(result["temperatures"])
            records.append({
                "City": result["city"],
                "Year": result["year"],
                "AvgMaxTemp": round(avg_temp, 2)
            })

SQLite

In [33]:
df = pd.DataFrame(records)
print(df.head())  # sanity check

# Save to SQLite
conn = sqlite3.connect("../climate_data.db")
df.to_sql("avg_temperatures", conn, if_exists="replace", index=False)
conn.close()


     City  Year  AvgMaxTemp
0  Zurich  2014       14.83
1  Zurich  2015       14.78
2  Zurich  2016       13.97
3  Zurich  2017       14.46
4  Zurich  2018       15.89
